Imports

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import mean_poisson_deviance
from interpret import show
from interpret.glassbox import ExplainableBoostingRegressor

Configuration

In [ ]:
DATA_PATH_TRAIN = "DATA/learn.csv"
DATA_PATH_TEST = "DATA/test.csv"

FEAT_GLM = [
    "VehPowerGLM", "VehAgeGLM", "DrivAgeGLM", "BonusMalusGLM",
    "VehBrand", "VehGas", "DensityGLM", "Region", "AreaGLM"
]

FEAT_EBM = [
    "VehPower", "VehAge", "DrivAge", "BonusMalus",
    "VehBrand", "VehGas", "Density", "Region", "AreaGLM"
]

EBM_FEATURE_TYPES = [
    "continuous", "continuous", "continuous", "continuous",
    "nominal", "nominal", "continuous", "nominal", "continuous"
]

Helper Functions

In [ ]:
def edr_poisson(y_true, y_pred, nul_pred, weights):
    """Compute Explained Deviance Reductions based on Poisson deviance."""
    nul_dev = mean_poisson_deviance(y_true=y_true, y_pred=nul_pred, sample_weight=weights)
    mod_dev = mean_poisson_deviance(y_true=y_true, y_pred=y_pred, sample_weight=weights)
    return 1 - mod_dev / nul_dev

def get_prediction_breakdown(ebm, X, feature_names=None):
    """Calculates contribution of each feature to the final prediction."""
    if feature_names is None:
        feature_names = ebm.feature_names if hasattr(ebm, "feature_names") else [f"F{i}" for i in range(X.shape[1])]

    X_array = X.values if isinstance(X, pd.DataFrame) else np.array(X)
    sample_scores, breakdowns = [], []

    for sample_idx, sample in enumerate(X_array):
        intercept_val = float(ebm.intercept_) if isinstance(ebm.intercept_, (int, float, np.number)) else float(ebm.intercept_[0])
        score = intercept_val
        breakdown = {"sample_idx": sample_idx, "intercept": intercept_val, "features": {}}

        for term_idx, features in enumerate(ebm.term_features_):
            term_name = " x ".join([feature_names[f] for f in features])
            tensor_index = []
            feature_values = {}

            for f_idx in features:
                val = sample[f_idx]
                feature_values[feature_names[f_idx]] = val
                bins = ebm.bins_[f_idx][min(len(ebm.bins_[f_idx]) - 1, len(features) - 1)]

                is_missing = val is None or (isinstance(val, float) and np.isnan(val)) or (isinstance(val, str) and val.strip() == "")
                if is_missing:
                    bin_idx = 0
                elif isinstance(bins, dict):
                    v_str = str(int(val)) if isinstance(val, (int, float)) and val == int(val) else str(val)
                    bin_idx = bins.get(v_str, len(bins) + 1)
                else:
                    bin_idx = np.digitize(float(val), bins, right=False) + 1
                tensor_index.append(bin_idx)

            local_score = ebm.term_scores_[term_idx][tuple(tensor_index)]
            score += local_score
            breakdown["features"][term_name] = {"value": feature_values, "contribution": float(local_score)}

        breakdown["total_score"] = float(score)
        sample_scores.append(score)
        breakdowns.append(breakdown)

    predictions = np.exp(np.array(sample_scores))
    return predictions, breakdowns

def get_ebm_coefficients(ebm, feature_names=None):
    """Extracts all bins and their corresponding scores from an EBM model."""
    if feature_names is None:
        feature_names = ebm.feature_names
    
    coeffs = []
    for term_idx, features in enumerate(ebm.term_features_):
        term_name = feature_names[features[0]] if len(features) == 1 else " x ".join([feature_names[f] for f in features])
        scores = ebm.term_scores_[term_idx]
        
        if len(features) == 1:
            f_idx = features[0]
            bins = ebm.bins_[f_idx][0]
            if isinstance(bins, dict):
                for cat, b_idx in bins.items():
                    coeffs.append({"term": term_name, "bin_value": cat, "score": float(scores[b_idx])})
                coeffs.append({"term": term_name, "bin_value": "MISSING", "score": float(scores[0])})
            else:
                for b_idx in range(len(scores)):
                    label = "MISSING" if b_idx == 0 else (f"<= {bins[0]}" if b_idx == 1 else (f"> {bins[-1]}" if b_idx > len(bins) else "BIN"))
                    coeffs.append({"term": term_name, "bin_value": label, "score": float(scores[b_idx])})
        else:
            coeffs.append({"term": term_name, "bin_value": "Interaction Tensor", "score": np.mean(np.abs(scores))})
            
    return pd.DataFrame(coeffs)

def double_lift_fixed(y_m1, y_m2, y_act, weight=None, p_tile=10, model_names=("M1", "M2")):
    """Generates a Double Lift Chart comparing two models against actuals."""
    sns.set()
    df = pd.DataFrame({model_names[0]: y_m1, model_names[1]: y_m2, "actual": y_act})
    df["weight"] = weight if weight is not None else 1
    df["Ratio"] = df[model_names[0]] / df[model_names[1]]
    df = df.sort_values("Ratio")
    df["cum_w"] = df["weight"].cumsum()
    df["p_tile"] = pd.qcut(df["cum_w"], p_tile, labels=False, duplicates="drop") + 1
    
    agg = df.groupby("p_tile").agg({model_names[0]: "mean", model_names[1]: "mean", "actual": "mean"}).reset_index()
    melted = agg.melt("p_tile", var_name="Model", value_name="Value")
    
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=melted, x="p_tile", y="Value", hue="Model", marker="o")
    plt.title("Double Lift Chart")
    plt.grid(True, alpha=0.3)
    return agg, df

Data Loading & Preprocessing

In [ ]:
train = pd.read_csv(DATA_PATH_TRAIN)
test = pd.read_csv(DATA_PATH_TEST)

train_expo, test_expo = train["Exposure"], test["Exposure"]
log_expo_train, log_expo_test = np.log(train_expo), np.log(test_expo)

y_train, y_test = train["ClaimNb"], test["ClaimNb"]
X_train_ebm, X_test_ebm = train[FEAT_EBM], test[FEAT_EBM]

GLM Base Model

In [ ]:
formula1 = "ClaimNb ~ C(VehPowerGLM) + C(VehAgeGLM) + C(DrivAgeGLM) + BonusMalusGLM + C(VehBrand) + C(VehGas) + DensityGLM + C(Region) + C(AreaGLM)"
glm1 = sm.GLM.from_formula(formula1, data=train, family=sm.families.Poisson(), offset=log_expo_train).fit()

GLM Complex Model

In [ ]:
formula3 = """ClaimNb ~ C(VehPowerGLM) + C(VehAgeGLM) + np.log(DrivAge) + I(DrivAge**3) + I(DrivAge**4) + 
              BonusMalusGLM * DrivAge + BonusMalusGLM * I(DrivAge**2) + C(VehBrand) + C(VehGas) + 
              DensityGLM + C(Region) + AreaGLM"""
glm3 = smf.glm(formula3, data=train, family=sm.families.Poisson(), offset=log_expo_train).fit()

EBM Models

In [ ]:
ebm_std = ExplainableBoostingRegressor(feature_names=FEAT_EBM, feature_types=EBM_FEATURE_TYPES, objective="poisson_deviance", n_jobs=1)
ebm_std.fit(X_train_ebm, y_train, init_score=log_expo_train)

ebm_best = ExplainableBoostingRegressor(feature_names=FEAT_EBM, feature_types=EBM_FEATURE_TYPES, objective="poisson_deviance", interactions=1, n_jobs=1)
ebm_best.fit(X_train_ebm, y_train, init_score=log_expo_train)

ebm_best2 = ExplainableBoostingRegressor(feature_names=FEAT_EBM, feature_types=EBM_FEATURE_TYPES, objective="poisson_deviance", interactions=3, n_jobs=1)
ebm_best2.fit(X_train_ebm, y_train, init_score=log_expo_train)

Evaluation

In [ ]:
preds_dict = {
    "GLM_Base": glm1.predict(test, offset=log_expo_test),
    "GLM_Complex": glm3.predict(test, offset=log_expo_test),
    "EBM_Std": ebm_std.predict(X_test_ebm, init_score=log_expo_test),
    "EBM_Best": ebm_best.predict(X_test_ebm, init_score=log_expo_test),
    "EBM_Best2": ebm_best2.predict(X_test_ebm, init_score=log_expo_test)
}

for name, p in preds_dict.items():
    dev = mean_poisson_deviance(y_test, p)
    print(f"{name} Test Deviance: {dev:.6f}")

Visualization & Explainability

In [ ]:
ebm_global = ebm_best2.explain_global()
show(ebm_global)

double_lift_fixed(preds_dict["EBM_Std"], preds_dict["EBM_Best2"], y_test, p_tile=20, model_names=("EBM_std", "EBM_best"))
plt.show()

X_sample = test[FEAT_EBM].head(10)
local_preds, breakdowns = get_prediction_breakdown(ebm_best, X_sample, feature_names=FEAT_EBM)
print("\nSample 0 Breakdown:", breakdowns[0])

df_coef = get_ebm_coefficients(ebm_best, FEAT_EBM)
df_coef.to_excel("ebm_coefficients.xlsx", index=False)
print("\nCoefficients exported to ebm_coefficients.xlsx")